In [ ]:
import sys  
sys.path.insert(0, '../search-engine')
from utils import *
import pandas as pd
from igraph import *

In [38]:
# get data 
data = get_tweets(None, None, mode = "read", data_directory = '../data/')

In [152]:
# since igraph does not use node ids -> we need to create a dict storing the conversion
ids_mapping = {}
counter = 0
for tweet in data:
    if "retweeted_status" in tweet.keys():
        if tweet["user"]["id"] not in ids_mapping.keys():
            ids_mapping[tweet["user"]["id"]] = counter
            counter += 1

In [153]:
retweets = {}
for i, tweet in enumerate(data): 
    try: 
        id2 = ids_mapping[tweet["retweeted_status"]["user"]["id"]]        
        id1 = ids_mapping[tweet["user"]["id"]]
        try: retweets[id1].append(id2)
        except: retweets[id1] = [id2]
    except: 
        continue

In [154]:
edgelist = []
for node in retweets:
    for i in retweets[node]:
        edgelist.append((node, i))

In [160]:
g = Graph(n = len(ids_mapping))
g.add_edges(edgelist)

In [ ]:
plot(g, layout = g.layout("kk"))

In [ ]:
# node -> users
# directed edge -> u->v: u retweeted at least once v